In [1]:
import pandas as pd
import json
import re
import src.cleaning as cl

In [212]:
df = pd.read_csv("./data/shots_v1.csv")

In [246]:
df.head()

,name,jornada,shot,coord_x,coord_y
0,J. Zyskowski,1,#local-in,24.750000,126.416664
1,J. Zyskowski,1,#local-in,24.750000,23.883333
2,J. Zyskowski,1,#local-out,25.705345,23.883333
3,J. Zyskowski,1,#local-in,27.616035,137.866669
4,J. Zyskowski,1,#local-in,27.128998,129.816666


In [214]:
df.rename(columns = {"0": "Jornada"}, inplace = True)
df.drop(columns = ["Unnamed: 0"], inplace = True)

In [215]:
names = cl.shots_per_player(df)

In [217]:
for elem in names:
    if elem == " ":
        names.remove(elem)

In [218]:
len(names)

37285

In [219]:
jornadas = cl.cumulat_shots(df)

In [221]:
len(jornadas)

33

In [222]:
# As we had problems trying to append the first gameday number of shots, we append it before looping. 
shots_per_jornada = [1157]
for i in range(len(jornadas)-1):
    shots_per_jornada.append(jornadas[i+1]-jornadas[i])

In [223]:
shots_per_jornada

[1157,
 1133,
 1141,
 1133,
 1133,
 1125,
 1135,
 1109,
 1112,
 1101,
 1146,
 1110,
 1107,
 1163,
 1173,
 1138,
 1051,
 1116,
 1140,
 1145,
 1158,
 985,
 1149,
 1157,
 1132,
 1159,
 1128,
 1141,
 1138,
 1147,
 1136,
 1166,
 1121]

In [224]:
num_jornada = list(range(1, 35))
num_jornada = [str(x) for x in num_jornada]

In [250]:
final_jornada = sum([[s] * n for s, n in zip(num_jornada, shots_per_jornada)], [])

In [252]:
#Finally we get a list with values from 1 to 34 (gamedays played) according to when every shot was made. 
len(final_jornada)

37285

### Now we are gonna divide the list "prueba" in order to get data for each three elements. 

In [227]:
#Our variable "prueba" contains all the information about shots every three values. 
len(prueba)

111855

### Here we divide the information contained in "prueba" in three lists. One to know if the shot was made or not, as a local or as a visitor. Another for its coordinate x and the last one for its coordinate y.

In [228]:
made = []
for i in range(0, len(prueba), 3):
    made.append(prueba[i])
    
coord_x = []
for i in range(1, len(prueba), 3):
    coord_x.append(prueba[i])
    
coord_y = []
for i in range(2, len(prueba), 3):
    coord_y.append(prueba[i])
    
cositas = list(zip(names, final_jornada, made, coord_x, coord_y))
df = pd.DataFrame(cositas)
df.rename(columns = {0: "name", 1: "jornada", 2: "shot", 3: "coord_x", 4: "coord_y"}, inplace = True)

### Aquí ya tenemos el dataset mínimamente decente. Falta cambiar coordenadas x de visitors, poner nombres bien y agrupar bien

In [254]:
df.head()

,name,jornada,shot,coord_x,coord_y
0,J. Zyskowski,1,#local-in,24.750000,126.416664
1,J. Zyskowski,1,#local-in,24.750000,23.883333
2,J. Zyskowski,1,#local-out,25.705345,23.883333
3,J. Zyskowski,1,#local-in,27.616035,137.866669
4,J. Zyskowski,1,#local-in,27.128998,129.816666


In [235]:
df["coord_x"] = pd.to_numeric(df["coord_x"], downcast="float")
df["coord_y"] = pd.to_numeric(df["coord_y"], downcast="float")
df["jornada"] = pd.to_numeric(df["jornada"], downcast="integer")

In [236]:
df_visitor = df.loc[(df["shot"] == "#visitor-in") | (df["shot"] == "#visitor-out") | (df["shot"] == "#visitor-dunk")]
df_local = df.loc[(df["shot"] == "#local-in") | (df["shot"] == "#local-out") | (df["shot"] == "#local-dunk")]
df_visitor

,name,jornada,shot,coord_x,coord_y
58,A. Doornekamp,1,#visitor-in,418.221130,35.733334
59,A. Doornekamp,1,#visitor-out,473.481323,134.466660
60,A. Doornekamp,1,#visitor-in,355.336945,203.533340
61,A. Doornekamp,1,#visitor-in,458.720306,259.466675
62,A. Doornekamp,1,#visitor-out,432.513855,249.300003
...,...,...,...,...,...
37280,T. Abromaitis,33,#visitor-out,454.243286,241.250000
37281,T. Abromaitis,33,#visitor-in,465.201660,165.216660
37282,T. Abromaitis,33,#visitor-in,471.683014,259.200012
37283,T. Abromaitis,33,#visitor-in,466.288116,158.116669


In [237]:
df_local

,name,jornada,shot,coord_x,coord_y
0,J. Zyskowski,1,#local-in,24.750000,126.416664
1,J. Zyskowski,1,#local-in,24.750000,23.883333
2,J. Zyskowski,1,#local-out,25.705345,23.883333
3,J. Zyskowski,1,#local-in,27.616035,137.866669
4,J. Zyskowski,1,#local-in,27.128998,129.816666
...,...,...,...,...,...
37218,A. Peters,33,#local-in,40.091721,150.550003
37219,A. Peters,33,#local-in,40.091721,129.183334
37220,A. Peters,33,#local-out,53.560219,167.133331
37221,A. Peters,33,#local-in,144.018311,211.366669


In [238]:
df_visitor["coord_x"] = abs(df_visitor["coord_x"] - 513)

<ipython-input-238-7949a2647ac1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_visitor["coord_x"] = abs(df_visitor["coord_x"] - 513)


In [239]:
df_visitor.dtypes

name        object
jornada       int8
shot        object
coord_x    float32
coord_y    float32
dtype: object

In [240]:
df = pd.concat([df_local, df_visitor])

In [241]:
df["name"] = df["name"].str.strip()

In [242]:
df

,name,jornada,shot,coord_x,coord_y
0,J. Zyskowski,1,#local-in,24.750000,126.416664
1,J. Zyskowski,1,#local-in,24.750000,23.883333
2,J. Zyskowski,1,#local-out,25.705345,23.883333
3,J. Zyskowski,1,#local-in,27.616035,137.866669
4,J. Zyskowski,1,#local-in,27.128998,129.816666
...,...,...,...,...,...
37280,T. Abromaitis,33,#visitor-out,58.756714,241.250000
37281,T. Abromaitis,33,#visitor-in,47.798340,165.216660
37282,T. Abromaitis,33,#visitor-in,41.316986,259.200012
37283,T. Abromaitis,33,#visitor-in,46.711884,158.116669


In [243]:
df["name"].str.contains("J. Dee").value_counts()

False    37032
True       253
Name: name, dtype: int64

In [245]:
df.to_csv("./data/database_shots.csv")